# LangGraph Advanced – Part 6️⃣: Fan‑out / Fan‑in Pattern

Demo chạy song song 3 truy vấn rồi gộp kết quả.

In [ ]:
%pip install -q langgraph langchain langchain-anthropic tiktoken

In [ ]:
from langgraph.graph import StateGraph, MessageState
from concurrent.futures import ThreadPoolExecutor
from langchain_anthropic import ChatAnthropic
from langchain.schema import HumanMessage

llm = ChatAnthropic(model_name="claude-3-haiku-20240307",temperature=0.4)

class FanState(MessageState):
    question:str

def search(engine:str):
    return f"Kết quả từ {engine}"

def fanout(state:FanState):
    engines=["wiki","news","books"]
    with ThreadPoolExecutor(max_workers=3) as pool:
        results=list(pool.map(search, engines))
    state.results=results
    return state

def merge(state:FanState):
    prompt="\n".join(state.results)
    state.answer=llm([HumanMessage(content=f"Tổng hợp:\n{prompt}")]).content
    print(state.answer)
    return state

g=StateGraph(FanState).add_node("fan",fanout).add_node("merge",merge)
g.set_entry_point("fan").add_edge("fan","merge")
g.compile().invoke(FanState(question="AI là gì?"))